# Naive Bayes Image Classifier

## Import Libraries

In [108]:
import os
import numpy as np
from PIL import Image
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage import exposure
from skimage.feature import graycomatrix, graycoprops
import matplotlib.pyplot as plt
from scipy.ndimage import shift
from scipy.ndimage import convolve

def pad_array(arr, n):
    #if len(arr) >= n:
        #return arr[:n]  # Trim the array if it's longer than n
    #else:
    pad_width = (0, n - len(arr))  # Calculate the padding width
    return np.pad(arr, pad_width, mode='constant', constant_values=0)


def rgb2gray(image):
    return np.dot(image[..., :3], [0.2989, 0.587, 0.114])

def sobel_filter(image):
    ## Standard Sobel
    kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    kernel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    ## Customer kernel
#     kernel_x = np.array([[0, 0, 0], [1, 0, -1], [0, 0, 0]])
#     kernel_y = np.array([[0, 1, 0], [0, 0, 0], [0, -1, 0]])
    
    gradient_x = convolve2d(image, kernel_x)
    gradient_y = convolve2d(image, kernel_y)
    
    edge_features = np.sqrt(gradient_x ** 2 + gradient_y ** 2)
    return edge_features

def sharpen_image(image, strength=1):
    # Create a Laplacian kernel for image sharpening
    kernel = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
    
    # Apply the convolution operation using the Laplacian kernel
    sharpened_image = image + strength * convolve(image, kernel)
    
    # Clip the pixel values to ensure they are within the valid range
    sharpened_image = np.clip(sharpened_image, 0, 255)
    
    return sharpened_image

def emboss_image(image):
    kernel = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
    
    # Apply the convolution operation using the emboss kernel
    embossed_image = convolve(image, kernel)
    
    # Clip the pixel values to ensure they are within the valid range
    embossed_image = np.clip(embossed_image, 0, 255)
    
    return embossed_image

def gaussian_filter(image, sigma=1):
    size = int(2 * np.ceil(3 * sigma) + 1)  # Filter size
    kernel = gaussian_kernel(size, sigma)  # Generate the Gaussian kernel
    
    # Apply the convolution operation using the Gaussian kernel
    filtered_image = convolve(image, kernel)
    
    return filtered_image


def gaussian_kernel(size, sigma):
    kernel = np.fromfunction(lambda x, y: gaussian(x, y, size, sigma), (size, size))
    kernel /= np.sum(kernel)  # Normalize the kernel
    
    return kernel


def gaussian(x, y, size, sigma):
    center = size // 2
    exponent = -((x - center) ** 2 + (y - center) ** 2) / (2 * sigma ** 2)
    return (1 / (2 * np.pi * sigma ** 2)) * np.exp(exponent)

def convolve2d(image, kernel):
    kernel_size = kernel.shape[0]
    padding = kernel_size // 2
    
    image_padded = np.pad(image, padding, mode='constant')
    height, width = image.shape
    
    output = np.zeros_like(image)
    
    for i in range(height):
        for j in range(width):
            patch = image_padded[i:i+kernel_size, j:j+kernel_size]
            output[i, j] = np.sum(patch * kernel)
    
    return output

def calculate_histogram(image):
    hist = np.zeros(256)
    height, width = image.shape
    
    for i in range(height):
        for j in range(width):
            intensity = image[i, j]
            hist[intensity] += 1
    
    return hist


def calculate_glcm(image):
    glcm = np.zeros((256, 256))
    height, width = image.shape

    for i in range(height - 1):
        for j in range(width - 1):
            intensity1 = image[i, j]
            intensity2 = image[i, j + 1]
            glcm[intensity1, intensity2] += 1
            glcm[intensity2, intensity1] += 1  # Add the symmetric entry
    
    glcm /= np.sum(glcm)
    return glcm

def calculate_contrast(glcm):
    contrast = 0
    for i in range(glcm.shape[0]):
        for j in range(glcm.shape[1]):
            contrast += glcm[i, j] * (i - j) ** 2
    return contrast

def calculate_energy(glcm):
    energy = np.sum(glcm ** 2)
    return energy


def calculate_correlation(glcm):
    row_mean = np.mean(glcm, axis=1, keepdims=True)
    col_mean = np.mean(glcm, axis=0, keepdims=True)
    row_std = np.std(glcm, axis=1, ddof=1, keepdims=True)
    col_std = np.std(glcm, axis=0, ddof=1, keepdims=True)
    
    correlation = np.sum((glcm - row_mean) * (glcm - col_mean)) / (row_std * col_std)
    return correlation

def extract_features(image, output_folder, file_name):
    # Convert the image to grayscale
    gray_image = rgb2gray(image)
    
    gray_image1 = gray_image
    
    # Convert the image to unsigned integer type
    gray_image = (gray_image * 255).astype(np.uint8)
    
    # Save the grayscale image
    grayscale_path = os.path.join(output_folder, file_name+'grayscale.png')
    plt.imsave(grayscale_path, gray_image1, cmap='gray')
    
    # Extract edge features using Sobel filter
    edge_features = sobel_filter(gray_image1)    
    
    # Save the edge features image
    edge_path = os.path.join(output_folder, file_name+'edge_features.png')
    plt.imsave(edge_path, edge_features, cmap='gray')
    
    # Extract sharpened features using Sobel filter
    sharpened_features = sharpen_image(gray_image1, 2)
    
    # Save the sharpened features image
    sharpened_path = os.path.join(output_folder, file_name+'sharpened_features.png')
    plt.imsave(sharpened_path, sharpened_features, cmap='gray')
    
    
    # Extract emboss features using filter
    emboss_features = emboss_image(gray_image1)
    
    # Save the emboss features image
    emboss_path = os.path.join(output_folder, file_name+'emboss_features.png')
    plt.imsave(emboss_path, emboss_features, cmap='gray')
    
    # Extract Gaussian features using filter
    gaussian_features = gaussian_filter(gray_image1)
    
    # Save the gaussian features image
    gaussian_path = os.path.join(output_folder, file_name+'gaussian_features.png')
    plt.imsave(gaussian_path, gaussian_features, cmap='gray')
    
    # Calculate histogram features
    hist = calculate_histogram(gray_image)
    
    # Calculate texture features using GLCM (Grey-Level Co-occurrence Matrix)
    glcm = calculate_glcm(gray_image)
    contrast = calculate_contrast(glcm)
    energy = calculate_energy(glcm)
    correlation = calculate_correlation(glcm)

    # Save the histogram features plot
    hist_path = os.path.join(output_folder, file_name+'histogram.png')
    plt.plot(hist)
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')
    plt.savefig(hist_path)
    plt.close()
    
    # Concatenate all features into a single feature vector
    #features = np.concatenate([edge_features.flatten(), gaussian_features.flatten(), sharpened_features.flatten(), emboss_features.flatten(), hist, [contrast, energy, correlation]])
    features = np.concatenate([edge_features.flatten(), sharpened_features.flatten(), emboss_features.flatten(), hist])

    return features

## Data Preprocessing

In [109]:
# Set the path to the image folders
folder_path = '../deeplearning-model/data/known_images/'
output_folder = '../deeplearning-model/data/known_images_out/'

# Set the target image size
target_size = (256, 256)  # Adjust this as needed

# Load the image data and labels
X = []
y = []
class_labels = ['cassava', 'maize', 'banana', 'weed']
#class_labels=['x']
for label in class_labels:
    folder = os.path.join(folder_path, label)
    images = os.listdir(folder)
    for image_file in images:
        img_path = os.path.join(folder, image_file)
        img = Image.open(img_path)
        img = img.convert('RGB')  # Convert to RGB if needed
        img = img.resize(target_size)  # Resize the image
                
        # Extract the features from the image
        img_features = extract_features(np.array(img), output_folder, label+image_file)
        flattened_img_features = pad_array(img_features.flatten(), 250000)
        
        #print(len(flattened_img_features))
        
        #img_array = np.array(img)
        #X.append(img_array.flatten())  # Flatten the image to a 1D vector
        X.append(flattened_img_features)
        y.append(label)
        #break
    #break

# Convert data to NumPy arrays
X = np.array(X)
y = np.array(y)

In [110]:
print(X.shape)

(133, 250000)


In [100]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
model = GaussianNB()
model.fit(X_train.reshape(X_train.shape[0], -1), y_train)
#model.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
#y_pred = model.predict(X_test)



ValueError: setting an array element with a sequence.

## Evaluate Model

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred, zero_division=1)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report)